In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import accelerate

In [4]:

# Define the tag categories that can be related to the movie dataset 
tags = ["revenue", "director", "actor", "release_date", "genre", "rating", "budget",
        "producer", "screenwriter", "birth_date", "birth_place"] 

In [5]:
data = [
    {"query": "what was the earning of the movie Titanic", "tag": "revenue"},
    {"query": "how much did Avatar make", "tag": "revenue"},
    {"query": "box office collection of Jurassic Park", "tag": "revenue"},
    {"query": "total revenue of Inception", "tag": "revenue"},
    {"query": "what was the gross of Frozen", "tag": "revenue"},
    {"query": "how much did Avengers Endgame earn", "tag": "revenue"},
    {"query": "earnings of The Dark Knight", "tag": "revenue"},
    {"query": "how much money did Titanic gross", "tag": "revenue"},
    {"query": "how much revenue did Star Wars make", "tag": "revenue"},
    {"query": "revenue of Iron Man", "tag": "revenue"},

    {"query": "who directed Titanic", "tag": "director"},
    {"query": "who was the director of Inception", "tag": "director"},
    {"query": "who directed the movie Avatar", "tag": "director"},
    {"query": "who directed Star Wars", "tag": "director"},
    {"query": "who was the director of The Godfather", "tag": "director"},
    {"query": "director of The Dark Knight", "tag": "director"},
    {"query": "who directed Forrest Gump", "tag": "director"},
    {"query": "who is the director of Jurassic Park", "tag": "director"},
    {"query": "who directed Schindler's List", "tag": "director"},
    {"query": "director of The Matrix", "tag": "director"},

    {"query": "who played the lead role in Titanic", "tag": "actor"},
    {"query": "who acted in The Dark Knight", "tag": "actor"},
    {"query": "lead actor in Inception", "tag": "actor"},
    {"query": "who starred in Avatar", "tag": "actor"},
    {"query": "who was the main actor in The Godfather", "tag": "actor"},
    {"query": "lead actor of Iron Man", "tag": "actor"},
    {"query": "who played in The Matrix", "tag": "actor"},
    {"query": "who was the actor in Star Wars", "tag": "actor"},
    {"query": "who acted in Interstellar", "tag": "actor"},
    {"query": "who starred in Pulp Fiction", "tag": "actor"},

    {"query": "when was Titanic released", "tag": "release_date"},
    {"query": "release date of Avatar", "tag": "release_date"},
    {"query": "when was Inception released", "tag": "release_date"},
    {"query": "release year of Star Wars", "tag": "release_date"},
    {"query": "when did The Dark Knight come out", "tag": "release_date"},
    {"query": "when was Schindler's List released", "tag": "release_date"},
    {"query": "release date of The Matrix", "tag": "release_date"},
    {"query": "release year of The Godfather", "tag": "release_date"},
    {"query": "when did Forrest Gump release", "tag": "release_date"},
    {"query": "when was Jurassic Park released", "tag": "release_date"},

    {"query": "what genre is Titanic", "tag": "genre"},
    {"query": "genre of Inception", "tag": "genre"},
    {"query": "what type of movie is The Godfather", "tag": "genre"},
    {"query": "what genre is Star Wars", "tag": "genre"},
    {"query": "what genre is The Dark Knight", "tag": "genre"},
    {"query": "genre of Avatar", "tag": "genre"},
    {"query": "what type of movie is Jurassic Park", "tag": "genre"},
    {"query": "what genre is The Matrix", "tag": "genre"},
    {"query": "what type of movie is Schindler's List", "tag": "genre"},
    {"query": "genre of Pulp Fiction", "tag": "genre"},

    {"query": "what is the rating of Titanic", "tag": "rating"},
    {"query": "rating of Inception", "tag": "rating"},
    {"query": "how was Star Wars rated", "tag": "rating"},
    {"query": "rating of The Dark Knight", "tag": "rating"},
    {"query": "what rating does The Matrix have", "tag": "rating"},
    {"query": "rating of Avatar", "tag": "rating"},
    {"query": "what rating does The Godfather have", "tag": "rating"},
    {"query": "rating of Forrest Gump", "tag": "rating"},
    {"query": "how was Pulp Fiction rated", "tag": "rating"},
    {"query": "what is the rating of Schindler's List", "tag": "rating"},

    {"query": "what was the budget of Titanic", "tag": "budget"},
    {"query": "budget of Inception", "tag": "budget"},
    {"query": "how much was spent on Avatar", "tag": "budget"},
    {"query": "budget for The Dark Knight", "tag": "budget"},
    {"query": "how much did Star Wars cost", "tag": "budget"},
    {"query": "what was the budget of The Godfather", "tag": "budget"},
    {"query": "how much did The Matrix cost", "tag": "budget"},
    {"query": "budget for Jurassic Park", "tag": "budget"},
    {"query": "how much did Pulp Fiction cost", "tag": "budget"},
    {"query": "budget of Forrest Gump", "tag": "budget"},

    {"query": "who produced Titanic", "tag": "producer"},
    {"query": "who was the producer of Inception", "tag": "producer"},
    {"query": "producer of Avatar", "tag": "producer"},
    {"query": "who produced The Dark Knight", "tag": "producer"},
    {"query": "who was the producer of Star Wars", "tag": "producer"},
    {"query": "who produced The Matrix", "tag": "producer"},
    {"query": "producer of The Godfather", "tag": "producer"},
    {"query": "who produced Jurassic Park", "tag": "producer"},
    {"query": "producer of Schindler's List", "tag": "producer"},
    {"query": "who produced Pulp Fiction", "tag": "producer"},

    {"query": "who wrote the screenplay for Titanic", "tag": "screenwriter"},
    {"query": "screenwriter of Inception", "tag": "screenwriter"},
    {"query": "who wrote the screenplay for The Dark Knight", "tag": "screenwriter"},
    {"query": "who wrote Star Wars", "tag": "screenwriter"},
    {"query": "who was the screenwriter of Avatar", "tag": "screenwriter"},
    {"query": "screenwriter of The Matrix", "tag": "screenwriter"},
    {"query": "who wrote The Godfather screenplay", "tag": "screenwriter"},
    {"query": "who wrote the script for Pulp Fiction", "tag": "screenwriter"},
    {"query": "who was the screenwriter of Schindler's List", "tag": "screenwriter"},
    {"query": "who wrote the screenplay for Jurassic Park", "tag": "screenwriter"},

    {"query": "when was Leonardo DiCaprio born", "tag": "birth_date"},
    {"query": "birth date of James Cameron", "tag": "birth_date"},
    {"query": "when was Robert Downey Jr born", "tag": "birth_date"},
    {"query": "birth date of Steven Spielberg", "tag": "birth_date"},
    {"query": "when was Christopher Nolan born", "tag": "birth_date"},
    {"query": "when was Al Pacino born", "tag": "birth_date"},
    {"query": "birth date of Keanu Reeves", "tag": "birth_date"},
    {"query": "when was Quentin Tarantino born", "tag": "birth_date"},
    {"query": "birth date of Harrison Ford", "tag": "birth_date"},
    {"query": "when was Samuel L. Jackson born", "tag": "birth_date"},

    {"query": "where was Leonardo DiCaprio born", "tag": "birth_place"},
    {"query": "birthplace of James Cameron", "tag": "birth_place"},
    {"query": "where was Robert Downey Jr born", "tag": "birth_place"},
    {"query": "where was Steven Spielberg born", "tag": "birth_place"},
    {"query": "birthplace of Christopher Nolan", "tag": "birth_place"},
    {"query": "where was Al Pacino born", "tag": "birth_place"},
    {"query": "where was Keanu Reeves born", "tag": "birth_place"},
    {"query": "birthplace of Quentin Tarantino", "tag": "birth_place"},
    {"query": "where was Harrison Ford born", "tag": "birth_place"},
    {"query": "where was Samuel L. Jackson born", "tag": "birth_place"},
]

In [7]:
# Map tags to numerical labels
tag2id = {tag: idx for idx, tag in enumerate(tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

# Prepare the dataset
queries = [d['query'] for d in data]
labels = [tag2id[d['tag']] for d in data]

# Split into train and test
train_queries, test_queries, train_labels, test_labels = train_test_split(queries, labels, test_size=0.2)

# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input data
train_encodings = tokenizer(train_queries, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_queries, truncation=True, padding=True, max_length=64)

# Convert to PyTorch dataset format
class MovieTagDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieTagDataset(train_encodings, train_labels)
test_dataset = MovieTagDataset(test_encodings, test_labels)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(tags))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Use the Trainer API to train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Test the model
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), axis=1)

# Map predicted labels back to tags
predicted_tags = [id2tag[label.item()] for label in predicted_labels]

# Print the results
for query, pred_tag in zip(test_queries, predicted_tags):
    print(f"Query: {query} --> Predicted Tag: {pred_tag}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda3/envs/atai/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▍         | 10/220 [00:03<01:01,  3.40it/s]

{'loss': 2.4565, 'grad_norm': 7.6797003746032715, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.91}


  5%|▌         | 11/220 [00:03<01:01,  3.40it/s]

{'eval_loss': 2.3814496994018555, 'eval_runtime': 0.1307, 'eval_samples_per_second': 168.288, 'eval_steps_per_second': 22.948, 'epoch': 1.0}


  9%|▉         | 20/220 [00:06<01:00,  3.32it/s]

...

{'loss': 0.293, 'grad_norm': 4.524197578430176, 'learning_rate': 2.2000000000000003e-05, 'epoch': 20.0}



100%|██████████| 220/220 [01:05<00:00,  3.36it/s]


{'eval_loss': 0.3136032819747925, 'eval_runtime': 0.0925, 'eval_samples_per_second': 237.877, 'eval_steps_per_second': 32.438, 'epoch': 20.0}
{'train_runtime': 65.5639, 'train_samples_per_second': 26.844, 'train_steps_per_second': 3.356, 'train_loss': 1.5786668864163484, 'epoch': 20.0}


100%|██████████| 3/3 [00:00<00:00, 57.95it/s]

Query: producer of Schindler's List --> Predicted Tag: producer
Query: who directed Titanic --> Predicted Tag: director
Query: what was the gross of Frozen --> Predicted Tag: rating
Query: who was the screenwriter of Avatar --> Predicted Tag: screenwriter
Query: how much did Star Wars cost --> Predicted Tag: budget
Query: how much did Pulp Fiction cost --> Predicted Tag: budget
Query: when was Titanic released --> Predicted Tag: release_date
Query: genre of Avatar --> Predicted Tag: genre
Query: birth date of Keanu Reeves --> Predicted Tag: birth_date
Query: who was the director of Inception --> Predicted Tag: director
Query: who produced Pulp Fiction --> Predicted Tag: producer
Query: who wrote the screenplay for Titanic --> Predicted Tag: screenwriter
Query: what was the earning of the movie Titanic --> Predicted Tag: revenue
Query: screenwriter of Inception --> Predicted Tag: screenwriter
Query: who directed Schindler's List --> Predicted Tag: director
Query: who produced Titanic --

In [8]:
id2tag

{0: 'revenue',
 1: 'director',
 2: 'actor',
 3: 'release_date',
 4: 'genre',
 5: 'rating',
 6: 'budget',
 7: 'producer',
 8: 'screenwriter',
 9: 'birth_date',
 10: 'birth_place'}

In [9]:
# Export the model file
model.save_pretrained("movie_tag_model")

In [10]:
# save the id2tag mapping
import json
with open('id2tag.json', 'w') as f:
    json.dump(id2tag, f)

